In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from datetime import datetime, timedelta
import calendar
import holidays
from dateutil.relativedelta import relativedelta

%matplotlib inline

In [3]:
#viz setup
# sns.set(style='whitegrid',font_scale=1.75,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))
sns.set(style='whitegrid',font_scale=1.5,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))

In [4]:
states = ["AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
regions = ["_ENC","_MAC","_MTN","_NEC","_PAC","PUS","_WNC","_WSC","_ESC","_SAC"]
sectors = ['RES','COM']

In [27]:
idx = 0
for state in states:
    for sector in sectors:
        print("{}-{}".format(state,sector))
        response_consumption = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.SALES.{}-{}.M".format(state,sector))
        j_consumption = response_consumption.json()
        tmp_consumption = pd.DataFrame(j_consumption['series'][0]['data'],columns=['month','sales_mkwh'])
        tmp_consumption['state'] = state
        tmp_consumption['sector'] = sector
        
        response_consumers = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.CUSTOMERS.{}-{}.M".format(state,sector))
        j_consumers = response_consumers.json()
        tmp_consumers = pd.DataFrame(j_consumers['series'][0]['data'],columns=['month','consumers'])
        tmp_consumers['state'] = state
        tmp_consumers['sector'] = sector
        
        response_price = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=ELEC.PRICE.{}-{}.M".format(state,sector))
        j_price = response_price.json()
        tmp_price = pd.DataFrame(j_price['series'][0]['data'],columns=['month','price'])
        tmp_price['state'] = state
        tmp_price['sector'] = sector
        
        tmp = tmp_consumption.merge(tmp_consumers,how='left',on=['month','state','sector']).merge(tmp_price,how='left',on=['month','state','sector'])
        
        if idx == 0:
            energy_data = tmp.copy()
        else:
            energy_data = energy_data.append(tmp)
        idx = idx +1

AL-RES
AL-COM
AZ-RES
AZ-COM
AR-RES
AR-COM
CA-RES
CA-COM
CO-RES
CO-COM
CT-RES
CT-COM
DC-RES
DC-COM
DE-RES
DE-COM
FL-RES
FL-COM
GA-RES
GA-COM
ID-RES
ID-COM
IL-RES
IL-COM
IN-RES
IN-COM
IA-RES
IA-COM
KS-RES
KS-COM
KY-RES
KY-COM
LA-RES
LA-COM
ME-RES
ME-COM
MD-RES
MD-COM
MA-RES
MA-COM
MI-RES
MI-COM
MN-RES
MN-COM
MS-RES
MS-COM
MO-RES
MO-COM
MT-RES
MT-COM
NE-RES
NE-COM
NV-RES
NV-COM
NH-RES
NH-COM
NJ-RES
NJ-COM
NM-RES
NM-COM
NY-RES
NY-COM
NC-RES
NC-COM
ND-RES
ND-COM
OH-RES
OH-COM
OK-RES
OK-COM
OR-RES
OR-COM
PA-RES
PA-COM
RI-RES
RI-COM
SC-RES
SC-COM
SD-RES
SD-COM
TN-RES
TN-COM
TX-RES
TX-COM
UT-RES
UT-COM
VT-RES
VT-COM
VA-RES
VA-COM
WA-RES
WA-COM
WV-RES
WV-COM
WI-RES
WI-COM
WY-RES
WY-COM


In [28]:
idx = 0
for region in regions:
    response_cool = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWCD{}.M".format(region))
    j_cool = response_cool.json()
    tmp_cool = pd.DataFrame(j_cool['series'][0]['data'],columns=['month','cooling_days'])
    tmp_cool['region'] = region
    
    response_heat = requests.get("http://api.eia.gov/series/?api_key=e45b817b9a5449da30e0b88815d5f119&series_id=STEO.ZWHD{}.M".format(region))
    j_heat = response_heat.json()
    tmp_heat = pd.DataFrame(j_heat['series'][0]['data'],columns=['month','heating_days'])
    tmp_heat['region'] = region
    
    tmp = tmp_cool.merge(tmp_heat,how='left',on=['month','region'])
    if idx == 0:
        heating_cooling_days = tmp.copy()
    else:
        heating_cooling_days = heating_cooling_days.append(tmp)
    idx = idx +1

In [29]:
energy_data['revenue'] = energy_data.sales_mkwh*energy_data.price
country = energy_data.groupby(['month','sector']).sum().reset_index()
country['state'] = 'USA'
country.price = country.revenue/country.sales_mkwh

In [30]:
energy_data = energy_data.append(country)

In [31]:
energy_data['use_per_capita'] = energy_data.sales_mkwh*1000000/energy_data.consumers

In [32]:
heating_cooling_days.region = [re.sub('_','',r) for r in heating_cooling_days.region]

In [33]:
states.extend(['USA'])

In [34]:
state_region_mapping = pd.DataFrame(data={'state': states})

In [35]:
state_region_mapping['region'] = ''

In [36]:
state_region_mapping.loc[state_region_mapping.state.isin(['WA','OR','CA']),'region'] = 'PAC'
state_region_mapping.loc[state_region_mapping.state.isin(['MT','ID','WY','NV','UT','CO','AZ','NM']),'region'] = 'MTN'
state_region_mapping.loc[state_region_mapping.state.isin(['ND','SD','MN','NE','IA','KS','MO']),'region'] = 'WNC'
state_region_mapping.loc[state_region_mapping.state.isin(['OK','TX','AR','LA']),'region'] = 'WSC'
state_region_mapping.loc[state_region_mapping.state.isin(['WI','IL','IN','MI','OH']),'region'] = 'ENC'
state_region_mapping.loc[state_region_mapping.state.isin(['KY','TN','MS','AL']),'region'] = 'ESC'
state_region_mapping.loc[state_region_mapping.state.isin(['WV','MD','DE','VA','NC','SC','GA','FL','DC']),'region'] = 'SAC'
state_region_mapping.loc[state_region_mapping.state.isin(['NY','PA','NJ']),'region'] = 'MAC'
state_region_mapping.loc[state_region_mapping.state.isin(['RI','CT','MA','NH','VT','ME']),'region'] = 'NEC'
state_region_mapping.loc[state_region_mapping.state.isin(['USA']),'region'] = 'PUS'

In [37]:
energy_data = energy_data.merge(state_region_mapping,how='left',on='state')

In [38]:
energy_data= energy_data.merge(heating_cooling_days,how='left',on=['month','region'])

In [39]:
energy_data = energy_data.dropna()

In [40]:
energy_data = pd.concat([energy_data,pd.get_dummies(energy_data.sector)],axis=1)

In [41]:
energy_data['time'] = [12*(int(d[0:4])-2008)+int(d[4:6]) for d in energy_data.month]

In [42]:
energy_data['year'] = [int(d[0:4]) for d in energy_data.month]
energy_data['mon'] = [int(d[4:6]) for d in energy_data.month]

In [43]:
def get_season(m):
    if (m == 12)|(m<=2):
        return 'winter'
    if (m>=3)&(m<=5):
        return 'spring'
    if(m>=6)&(m<=8):
        return 'summer'
    if(m>=9)&(m<=11):
        return 'fall'

In [44]:
energy_data['season'] = energy_data.mon.apply(get_season)

In [45]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,heating_days,COM,RES,time,year,mon,season
0,202101,3164.80449,AL,RES,2284207.0,12.38,39180.279586,1385.515625,ESC,5.622370,736.805664,0,1,157,2021,1,winter
1,202012,2992.76542,AL,RES,2326166.0,11.39,34087.598134,1286.565714,ESC,0.817961,728.098912,0,1,156,2020,12,winter
2,202011,2056.87325,AL,RES,2324212.0,12.99,26718.783518,884.976607,ESC,12.121163,344.122734,0,1,155,2020,11,fall
3,202010,2150.98005,AL,RES,2326554.0,13.17,28328.407259,924.534763,ESC,66.448161,156.199061,0,1,154,2020,10,fall
4,202009,2876.36566,AL,RES,2325509.0,13.18,37910.499399,1236.875738,ESC,209.894047,19.820836,0,1,153,2020,9,fall


In [46]:
energy_data['date'] = [datetime(y,m,1) for y,m in zip(energy_data.year, energy_data.mon)]

In [47]:
def get_datetime_features(date):
    st = date
    en = date + relativedelta(months=1) - relativedelta(days=1)
    
    ## number of days in month
    num_days = len(pd.date_range(st,en))
    ## number of weekends in month
    num_weekends = pd.date_range(st,en).weekday.isin([5,6]).sum()
    ## number of holidays in month
    us_holidays = holidays.US(years=date.year)
    us_holidays = pd.DataFrame(us_holidays.items(),columns=['date','hol'])
    us_holidays['date'] = pd.to_datetime(us_holidays.date)
    num_holidays = len(us_holidays[(us_holidays.date.dt.month == date.month) & ~(us_holidays.date.dt.weekday.isin([5,6]))])
    
    num_weekends_or_holidays = num_holidays+num_weekends
    ## % of weekdays in month
    pct_weekdays = 1 - (num_holidays+num_weekends)/num_days
    
    return num_days, num_weekends_or_holidays, pct_weekdays

In [48]:
energy_data['num_days'], energy_data['num_hols'], energy_data['pct_weekdays'] = zip(*energy_data.date.apply(get_datetime_features)) 

In [49]:
energy_data['y'] = energy_data.use_per_capita/energy_data.num_days

In [50]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,RES,time,year,mon,season,date,num_days,num_hols,pct_weekdays,y
0,202101,3164.80449,AL,RES,2284207.0,12.38,39180.279586,1385.515625,ESC,5.622370,...,1,157,2021,1,winter,2021-01-01,31,12,0.612903,44.694052
1,202012,2992.76542,AL,RES,2326166.0,11.39,34087.598134,1286.565714,ESC,0.817961,...,1,156,2020,12,winter,2020-12-01,31,9,0.709677,41.502120
2,202011,2056.87325,AL,RES,2324212.0,12.99,26718.783518,884.976607,ESC,12.121163,...,1,155,2020,11,fall,2020-11-01,30,11,0.633333,29.499220
3,202010,2150.98005,AL,RES,2326554.0,13.17,28328.407259,924.534763,ESC,66.448161,...,1,154,2020,10,fall,2020-10-01,31,10,0.677419,29.823702
4,202009,2876.36566,AL,RES,2325509.0,13.18,37910.499399,1236.875738,ESC,209.894047,...,1,153,2020,9,fall,2020-09-01,30,9,0.700000,41.229191


In [51]:
energy_data.tail()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,RES,time,year,mon,season,date,num_days,num_hols,pct_weekdays,y
24095,202011,98994.84172,USA,RES,136421078.0,13.287650,1.315409e+06,725.656498,PUS,32.134410,...,1,155,2020,11,fall,2020-11-01,30,11,0.633333,24.188550
24096,202012,103951.57447,USA,COM,18796159.0,10.419035,1.083075e+06,5530.468989,PUS,6.868902,...,0,156,2020,12,winter,2020-12-01,31,9,0.709677,178.402225
24097,202012,129037.22650,USA,RES,137531520.0,12.756682,1.646087e+06,938.237478,PUS,6.868902,...,1,156,2020,12,winter,2020-12-01,31,9,0.709677,30.265725
24098,202101,103755.63610,USA,COM,18733383.0,10.251220,1.063622e+06,5538.542403,PUS,9.916634,...,0,157,2021,1,winter,2021-01-01,31,12,0.612903,178.662658
24099,202101,136791.50557,USA,RES,136154434.0,12.643004,1.729456e+06,1004.679037,PUS,9.916634,...,1,157,2021,1,winter,2021-01-01,31,12,0.612903,32.409001


In [52]:
energy_data.date.max()

Timestamp('2021-01-01 00:00:00')

In [57]:
data_urls = ['https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2008_c20180718.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2009_c20180718.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2010_c20200922.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2011_c20180718.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2012_c20200317.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2013_c20170519.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2014_c20210120.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2015_c20191116.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2016_c20190817.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2017_c20210120.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2018_c20201216.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2019_c20210223.csv.gz',
            'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2020_c20210223.csv.gz']

In [58]:
idx=0
for d in data_urls:
    tmp = pd.read_csv(d)
    if idx == 0:
        storm_data = tmp.copy()
    else:
        storm_data = storm_data.append(tmp)
    idx = idx +1

In [295]:
storm_data.EVENT_TYPE.value_counts()

Thunderstorm Wind             211025
Hail                          146046
Flash Flood                    49644
Winter Weather                 47401
High Wind                      40517
Winter Storm                   40031
Drought                        36481
Flood                          35033
Heavy Snow                     29743
Marine Thunderstorm Wind       23447
Tornado                        18160
Heavy Rain                     17703
Strong Wind                    14037
Heat                           10576
Extreme Cold/Wind Chill         8864
Blizzard                        8810
Frost/Freeze                    8695
Dense Fog                       8654
Excessive Heat                  7893
Lightning                       6867
High Surf                       5879
Cold/Wind Chill                 5728
Funnel Cloud                    4721
Wildfire                        4588
Ice Storm                       3796
Tropical Storm                  3441
Waterspout                      2630
C

In [294]:
storm_data[storm_data.DAMAGE_PROPERTY == '629.00M']

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,deaths
34007,201608,12,415,201608,13,630,109514,656655,LOUISIANA,22,...,WSW,BAPTIST,30.999,-90.566,30.487,-90.567,A slow moving low pressure system brought wide...,Twelve to eighteen inches of rainfall over a 2...,CSV,2


In [362]:
# storm_data_clean = storm_data[storm_data.MAGNITUDE>60].copy()
storm_data_clean = storm_data[['BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH',
       'END_DAY', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'EVENT_TYPE','MAGNITUDE', 'CATEGORY', 'TOR_F_SCALE',
       'EPISODE_NARRATIVE']].copy()

In [363]:
storm_data_clean = storm_data_clean.drop_duplicates(subset=['EVENT_TYPE','EPISODE_ID','STATE'])

In [364]:
storm_data_clean.tail()

,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,EVENT_TYPE,MAGNITUDE,CATEGORY,TOR_F_SCALE,EPISODE_NARRATIVE
56960,202006,30,202006,30,148524,894635,ATLANTIC SOUTH,87,Waterspout,NaN,NaN,NaN,Southwest steering flow pushed a stronger west...
56961,202006,7,202006,7,148220,894216,FLORIDA,12,Flood,NaN,NaN,NaN,Deep tropical moisture was in place (PWAT 2.5)...
56972,202006,2,202006,2,148832,896638,KANSAS,20,Dust Storm,NaN,NaN,NaN,A weak landspout tornado was reported in Walla...
56973,202006,2,202006,2,148832,904958,KANSAS,20,Tornado,NaN,NaN,EFU,A weak landspout tornado was reported in Walla...
56979,202006,21,202006,21,149157,899161,WISCONSIN,55,Thunderstorm Wind,60.0,NaN,NaN,A cold front advancing out of the northwest in...


In [365]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'USA':'USA'
}

In [366]:
us_state_abbrev_caps = {k.upper():v.upper() for k,v in us_state_abbrev.items()}

In [367]:
storm_data_clean['state'] = storm_data_clean.STATE.map(us_state_abbrev_caps)

In [368]:
storm_data_clean = storm_data_clean[~storm_data_clean.state.isna()]
storm_data_clean = storm_data_clean.drop(columns='STATE')

In [369]:
storm_data_clean['begin_date'] = [str(y) + str(d).zfill(2) for y,d in zip(storm_data_clean.BEGIN_YEARMONTH,storm_data_clean.BEGIN_DAY)]
storm_data_clean['end_date'] = [str(y) + str(d).zfill(2) for y,d in zip(storm_data_clean.END_YEARMONTH,storm_data_clean.END_DAY)]

In [370]:
storm_data_clean.begin_date = pd.to_datetime(storm_data_clean.begin_date)
storm_data_clean.end_date = pd.to_datetime(storm_data_clean.end_date)

In [371]:
storm_data_clean = storm_data_clean.drop_duplicates(subset='EPISODE_NARRATIVE').sort_values(['state','begin_date'])

In [372]:
storm_data_clean['num_days'] = (storm_data_clean.end_date - storm_data_clean.begin_date)

In [373]:
storm_data_clean.num_days = storm_data_clean.num_days.dt.days +1

In [374]:
storm_data_clean = storm_data_clean.drop(columns=['BEGIN_DAY','END_YEARMONTH','END_DAY'])

In [375]:
# events_to_keep = ['Thunderstorm Wind', 'Hail', 'Flash Flood', 'Flood', 'High Wind',
#        'Winter Weather', 'Tornado', 'Winter Storm', 'Heavy Snow', 'Heavy Rain',
#        'Lightning', 'Strong Wind', 'Blizzard', 'Heat', 'Frost/Freeze',
#        'Extreme Cold/Wind Chill', 'Excessive Heat', 'Cold/Wind Chill',
#        'Lake-Effect Snow',
#        'Ice Storm','Tropical Storm', 'Freezing Fog', 
#        'Hurricane (Typhoon)', 
#        'Hurricane']
events_to_keep = [
       'Winter Weather', 'Winter Storm', 'Heavy Snow', 'Blizzard', 'Heat', 'Frost/Freeze',
       'Extreme Cold/Wind Chill', 'Excessive Heat', 'Cold/Wind Chill','Lake-Effect Snow','Ice Storm',
        'Thunderstorm Wind', 'High Wind','Tornado','Heavy Rain','Strong Wind','Tropical Storm', 'Hurricane (Typhoon)', 'Hurricane']
hot_cold_map = {'Winter Weather':'cold', 'Winter Storm':'cold', 'Heavy Snow':'cold', 'Blizzard':'cold', 'Heat':'hot', 'Frost/Freeze':'cold',
       'Extreme Cold/Wind Chill':'cold', 'Excessive Heat':'hot', 'Cold/Wind Chill':'cold','Lake-Effect Snow':'cold','Ice Storm':'cold',
               'Thunderstorm Wind':'wind', 'High Wind':'wind','Tornado':'wind','Heavy Rain':'wind','Strong Wind':'wind','Tropical Storm':'wind',
                'Hurricane (Typhoon)':'wind', 'Hurricane':'wind'}

In [376]:
storm_data_clean = storm_data_clean[storm_data_clean.EVENT_TYPE.isin(events_to_keep)]
storm_data_clean['hot_cold'] = storm_data_clean.EVENT_TYPE.map(hot_cold_map)

In [377]:
storm_data_clean = storm_data_clean.groupby(['state','BEGIN_YEARMONTH','hot_cold']).sum().reset_index()

In [378]:
storm_data_clean = storm_data_clean.rename(columns={'BEGIN_YEARMONTH':'month'})
storm_data_clean.month = storm_data_clean.month.astype(str)

In [379]:
storm_data_clean = storm_data_clean.pivot(index=['state','month'], columns='hot_cold', values='num_days').reset_index().fillna(0)

In [380]:
energy_data = energy_data.drop(columns=['hot','cold'])

In [382]:
energy_data = energy_data.merge(storm_data_clean[['state','month','hot','cold','wind']],how='left',on=['state','month'])
# energy_data.storm_days = energy_data.storm_days.fillna(0)

In [383]:
energy_data.hot = energy_data.hot.fillna(0)
energy_data.cold = energy_data.cold.fillna(0)
energy_data.wind = energy_data.wind.fillna(0)

In [384]:
energy_data.head()

,month,sales_mkwh,state,sector,consumers,price,revenue,use_per_capita,region,cooling_days,...,mon,season,date,num_days,num_hols,pct_weekdays,y,hot,cold,wind
0,202101,3164.80449,AL,RES,2284207.0,12.38,39180.279586,1385.515625,ESC,5.622370,...,1,winter,2021-01-01,31,12,0.612903,44.694052,0.0,0.0,0.0
1,202012,2992.76542,AL,RES,2326166.0,11.39,34087.598134,1286.565714,ESC,0.817961,...,12,winter,2020-12-01,31,9,0.709677,41.502120,0.0,0.0,0.0
2,202011,2056.87325,AL,RES,2324212.0,12.99,26718.783518,884.976607,ESC,12.121163,...,11,fall,2020-11-01,30,11,0.633333,29.499220,0.0,3.0,2.0
3,202010,2150.98005,AL,RES,2326554.0,13.17,28328.407259,924.534763,ESC,66.448161,...,10,fall,2020-10-01,31,10,0.677419,29.823702,0.0,0.0,6.0
4,202009,2876.36566,AL,RES,2325509.0,13.18,37910.499399,1236.875738,ESC,209.894047,...,9,fall,2020-09-01,30,9,0.700000,41.229191,0.0,0.0,3.0


In [385]:
energy_data.to_csv('energy_data.csv',index=False)